In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
HEADERS = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
}

In [3]:
file = open("url_list.txt", "r")
url_list = file.readlines()
file.close
url_list = [url.replace('\n', '') for url in url_list]

In [4]:
list_to_pandas = []

In [5]:
def page_parsing(url):
    print(url)
    print('Page parsing completed:', end=' ')
    for page in range(1, 18):
        link = requests.get(url, headers=HEADERS, params={'page': page})
        time.sleep(1)
        soap = BeautifulSoup(link.text, "html.parser")
        items = soap.find_all('div', class_ = "catalog-item__container-flex")
        for item in items:
            apartment = item.find('div', class_ = "catalog-item__headline").text.split(' ')
            if apartment[0] == 'Студия':
                type_apartment = 'Студия'
                area = float(apartment[1])
                rooms = 0
            if apartment[1] == 'квартира':
                type_apartment = 'Квартира'
                rooms = int(apartment[0].split('-')[0])
                area = float(apartment[2])
            if apartment[0] == 'Апарт-студия':
                type_apartment = 'Студия'
                area = float(apartment[1])
                rooms = 0
            if apartment[1] == 'апартаменты':
                type_apartment = 'Квартира'
                rooms = int(apartment[0].split('-')[0])
                area = float(apartment[2])
            params = item.find('div', class_ = "catalog-item__container-param").find_all('span', class_ = "catalog-item__param")
            living_space = kitchen_area = ceiling = floor = total_floors = walls = None
            for param in params:
                apartment_description = param.text.replace('\n', '').rstrip().split(' ')
                if apartment_description[-1] == 'жилая':
                    living_space = float(apartment_description[0])
                    continue
                if apartment_description[-1] == 'кухня':
                    kitchen_area = float(apartment_description[0])
                    continue
                if apartment_description[-1] == 'потолок':
                    ceiling = float(apartment_description[0])
                    continue
                if apartment_description[-1] == 'этаж':
                    floor = int(apartment_description[0].split('/')[0])
                    total_floors = int(apartment_description[0].split('/')[1])
                    continue
                walls = param.text.replace('\n', '')
            district = item.find('div', class_ = "catalog-item__district").text.split(' ')[0]
            metro = metro_distance = None
            if len(item.find_all('span', class_ = "catalog-item__metro-name")) != 0:
                metro = item.find('span', class_ = "catalog-item__metro-name").text.replace('\xa0', '')
            if len(item.find_all('span', class_ = "catalog-item__metro-distance")) != 0:
                metro_distance = float(item.find('span', class_ = "catalog-item__metro-distance").text.replace(' км', ''))
            price = int(item.find('div', class_ = "catalog-item__price").text.replace(' ', '').replace('₽', ''))
            list_to_pandas.append([type_apartment, price, rooms, area, living_space, kitchen_area, ceiling, floor, total_floors,
                                   walls, district, metro, metro_distance])
        print(page, end=' ')
    print()

In [6]:
for url in url_list:
    page_parsing(url)

https://www.bn.ru/kvartiry-vtorichka/
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka/?sorting=5
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka/?district[]=24
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-moskovskij-rayon/
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-centralnyj-rayon/
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-primorskij-rayon/
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-nevskij-rayon/
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-nevskij-rayon/?sorting=5
Page parsing completed: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
https://www.bn.ru/kvartiry-vtorichka-pushkinski

In [7]:
column = ['Тип', 'Цена (₽)', 'Комнаты', 'Площадь (м²)', 'Жилая площадь (м²)', 'Площадь кухни (м²)', 'Потолок', 'Этаж',
          'Всего этажей в доме', 'Стены', 'Район', 'Метро', 'Расстояние до метро (км)']

df = pd.DataFrame(list_to_pandas, columns = column)

In [9]:
df = df.drop_duplicates()
df = df.sample(frac=1)
df.to_csv('apartment_sales_spb.csv', encoding='utf-8-sig', index= False)

In [12]:
display(df)

,Тип,Цена (₽),Комнаты,Площадь (м²),Жилая площадь (м²),Площадь кухни (м²),Потолок,Этаж,Всего этажей в доме,Стены,Район,Метро,Расстояние до метро (км)
0,Квартира,15700000,2,68.8,30.70,19.70,2.70,21,21,Кирпичный,Выборгский,Озерки,1.3
1,Квартира,19999000,2,95.7,35.17,24.89,2.70,15,20,Индивидуальный,Красносельский,NaN,NaN
2,Квартира,5330000,2,41.2,26.20,7.00,NaN,5,5,Панельный,Калининский,NaN,NaN
3,Квартира,145000000,3,206.8,88.30,51.00,3.00,10,16,Кирпично-Монолитный,Выборгский,Площадь Ленина,0.7
4,Квартира,22500000,5,133.0,86.80,10.00,3.30,5,7,Ст.Фонд Кап.Рем.,Адмиралтейский,Технологический институт,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6333,Квартира,8700000,1,42.3,18.00,8.60,2.60,3,14,Кирпичный,Московский,Московская,0.4
6334,Квартира,13921000,3,68.6,47.10,10.40,2.55,3,26,Панельный,Невский,Проспект Большевиков,NaN
6335,Квартира,16000000,2,63.0,31.90,14.00,2.75,5,25,Индивидуальный,Калининский,Гражданский проспект,NaN
6336,Квартира,21800000,3,90.0,43.70,13.20,2.64,22,22,Индивидуальный,Красносельский,NaN,NaN
